In [1]:
from typing import List
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorizedQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType,
)
from azure.core.credentials import AzureKeyCredential
import os
import azure.identity
from dotenv import load_dotenv
load_dotenv()
from openai import AzureOpenAI

In [2]:
# Initialize Azure search variables
AZURE_SEARCH_ENDPOINT = os.getenv("CONTOSO_SEARCH_ENDPOINT")
AZURE_SEARCH_KEY = os.getenv("CONTOSO_SEARCH_KEY")

# Set up OpenAI client based on environment variables
AZURE_OPENAI_SERVICE = os.getenv("CONTOSO_AI_SERVICES_ENDPOINT")
AZURE_OPENAI_KEY = os.getenv("CONTOSO_AI_SERVICES_KEY")
AZURE_OPENAI_ADA_DEPLOYMENT = "embedding-ada"
index_name = "contoso-products"

openai_client = AzureOpenAI(
        api_version="2023-07-01-preview",
        azure_endpoint=AZURE_OPENAI_SERVICE,
        azure_deployment=AZURE_OPENAI_ADA_DEPLOYMENT,
        api_key=AZURE_OPENAI_KEY
    )

search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT, index_name=index_name, credential=AzureKeyCredential(AZURE_SEARCH_KEY))


In [3]:
def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

In [4]:
def retrieve_documentation(
    question: str,
    index_name: str,
    embedding: List[float],
) -> str:

    vector_query = VectorizedQuery(
        vector=embedding, k_nearest_neighbors=3, fields="contentVector"
    )

    results = search_client.search(
        search_text=question,
        vector_queries=[vector_query],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=3,
    )

    docs = [
        {
            "id": doc["id"],
            "title": doc["title"],
            "content": doc["content"],
            "url": doc["url"],
        }
        for doc in results
    ]

    return docs

In [10]:
question = "Can you tell me about your hiking jackets?"
embedding = get_embedding(question)

docs = retrieve_documentation(question, index_name, embedding)

for item in docs:
    print(item)

{'id': '17', 'title': 'RainGuard Hiking Jacket', 'content': "Introducing the MountainStyle RainGuard Hiking Jacket - the ultimate solution for weatherproof comfort during your outdoor undertakings! Designed with waterproof, breathable fabric, this jacket promises an outdoor experience that's as dry as it is comfortable. The rugged construction assures durability, while the adjustable hood provides a customizable fit against wind and rain. Featuring multiple pockets for safe, convenient storage and adjustable cuffs and hem, you can tailor the jacket to suit your needs on-the-go. And, don't worry about overheating during intense activities - it's equipped with ventilation zippers for increased airflow. Reflective details ensure visibility even during low-light conditions, making it perfect for evening treks. With its lightweight, packable design, carrying it inside your backpack requires minimal effort. With options for men and women, the RainGuard Hiking Jacket is perfect for hiking, ca